The task is to build a text classifier using Markov model.

In [1]:
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

In [2]:
# download datasets: poems by Edgar Allan Poe and Robert Frost
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [3]:
input_files = ['edgar_allan_poe.txt', 'robert_frost.txt'] 

In [4]:
# assign labels to texts 
input_texts = []
labels = []

for label, file in enumerate(input_files):
    print(f'{file} corresponds to label {label}')
    
    for line in open(file):
        line = line.rstrip().lower()
        if line:
            line = line.translate(str.maketrans('', '', string.punctuation))
            
            input_texts.append(line)
            labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [5]:
# split to train and test sets
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [6]:
# make a dictionary of words to indeces, also include unknown token for words that might be present only in one set 
idx = 1
word2idx = {'<unk>': 0}

for text in train_text:
    tokens = text.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx += 1

In [7]:
# convert texts into lists of integers
train_text_int = []
test_text_int = []

for text in train_text:
    tokens = text.split()
    line_as_int = [word2idx[token] for token in tokens]
    train_text_int.append(line_as_int)
    
for text in test_text:
    tokens = text.split()
    line_as_int = [word2idx.get(token, 0) for token in tokens]
    test_text_int.append(line_as_int)

In [8]:
# initialize A and pi matrices for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [9]:
# create a function to compute counts for A and pi
def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            if last_idx is None:
                pi[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

In [10]:
# apply the function to classes
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [11]:
# normalize A and pi so they become valid probability matrices
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [12]:
# apply log function on A and pi
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [13]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33126934984520123, 0.6687306501547987)

In [14]:
# build a classifier
class Classifier():
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) # number of classes
        
    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]
        
        last_idx = None
        logprob = 0
        
        for idx in input_:
            if last_idx is None:
                logprob += logpi[idx]
            else:
                logprob += logA[last_idx, idx]
            last_idx = idx
        
        return logprob
    
    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        
        for i, input_ in enumerate(inputs):
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)]
            pred = np.argmax(posteriors)
            predictions[i] = pred
            
        return predictions

In [15]:
# use classifier
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [16]:
# calculate accuracy
Ptrain = clf.predict(train_text_int)
print(f'Train accuracy: {np.mean(Ptrain == Ytrain)}')

Train accuracy: 0.9944272445820433


In [17]:
Ptest = clf.predict(test_text_int)
print(f'Test accuracy: {np.mean(Ptest == Ytest)}')

Test accuracy: 0.8144712430426716


In [18]:
# calculate confusion matrics and F1 score: the model seems to show good results
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 526,    9],
       [   0, 1080]])

In [19]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 90,  93],
       [  7, 349]])

In [20]:
f1_score(Ytrain, Ptrain)

0.995850622406639

In [21]:
f1_score(Ytest, Ptest)

0.87468671679198